## This is a RAG app using llamindex

In [4]:
%pip install -q cassio datasets langchain langchain_community openai tiktoken llama-index


In [5]:
%pip install PyPDF2

In [6]:
from PyPDF2 import PdfReader

In [7]:
!wget https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf

--2025-07-21 04:23:27--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 23.212.250.8, 23.212.250.7
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|23.212.250.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘2025-isg-outlook.pdf’

2025-isg-outlook.pd     [ <=>                ]   6.88M  --.-KB/s    in 0.1s    

2025-07-21 04:23:27 (62.9 MB/s) - ‘2025-isg-outlook.pdf’ saved [7209986]



In [8]:
%pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [9]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent

In [11]:
documents = SimpleDirectoryReader("./").load_data("/content/2025-isg-outlook.pdf")

Loading files:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

